## Gerando arquivo para NER-Genia

Teste com formato QA.. tem como retornar as entidades descontinuas tbm?

In [40]:
import os
from pathlib import Path
import re
import pickle
import random


In [41]:
def getTiposEntidade():
    return ['DNA','RNA','Cell_type','Cell_line','Protein']

In [42]:
def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

In [43]:
def save_obj(name, obj):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    with open('../obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    existeDir = os.path.exists('../obj')
    if not existeDir:
        os.makedirs('../obj')
    try:
        with open('../obj/' + name + '.pkl', 'rb') as f:
            return pickle.load(f)
    except:
        raise
        return None


In [44]:
import re

s = 'Make the World a 2.4Better Place2.0'
pattern = r'([0-9])\.([0-9])'
replacement = r'\1y\2'
html = re.sub(pattern, replacement, s)

print(html)

Make the World a 2y4Better Place2y0


In [45]:
dic_sentencesTest=load_obj('dicEntidadesTesteGenia')
dic_sentencesDev=load_obj('dicEntidadesDevGenia')
dic_sentencesTrain=load_obj('dicEntidadesTrainGenia')

In [46]:
len(dic_sentencesTrain)

15023

In [47]:
dic_sentencesTrain[0]

[' IL - 2 gene expression and NF - kappa B activation through CD28 requires reactive oxygen production by 5 - lipoxygenase .',
 [['IL - 2 gene', [0, 1, 2, 3], 'DNA'],
  ['NF - kappa B', [6, 7, 8, 9], 'protein'],
  ['CD28', [12], 'protein'],
  ['5 - lipoxygenase', [18, 19, 20], 'protein']]]

In [48]:
dic_sentencesTest[3]

[' Characterization of HB24 expression in lymphoid and select developing tissues was performed by in situ hybridization .',
 [['HB24', [2], 'DNA']]]

In [49]:
def arrumaDic(dic_sentences):
    dic_sentences2={}
    for key, value in dic_sentences.items():
        #print('key:', key)
        #print('value:', value)
        tokens=value[0]
        #print('tokens:', tokens)
        #listaTokens=[[t, i] for i, t in enumerate(tokens.strip().split(' '))]
        listaTokens=[]
        ultimoIndice=0
        for i, t in enumerate(tokens.strip().split(' ')):
            inicio=ultimoIndice
            listaTokens.append([t, i, inicio])
            ultimoIndice=ultimoIndice+len(t)+1

        entidades=value[1]
        valores=[listaTokens, entidades]
        dic_sentences2[key]=valores
        #break
    return dic_sentences2

dic_sentencesTest2 = arrumaDic(dic_sentencesTest)
dic_sentencesTrain2 = arrumaDic(dic_sentencesTrain)
dic_sentencesDev2 = arrumaDic(dic_sentencesDev)

dic_sentencesTrain2[0]

[[['IL', 0, 0],
  ['-', 1, 3],
  ['2', 2, 5],
  ['gene', 3, 7],
  ['expression', 4, 12],
  ['and', 5, 23],
  ['NF', 6, 27],
  ['-', 7, 30],
  ['kappa', 8, 32],
  ['B', 9, 38],
  ['activation', 10, 40],
  ['through', 11, 51],
  ['CD28', 12, 59],
  ['requires', 13, 64],
  ['reactive', 14, 73],
  ['oxygen', 15, 82],
  ['production', 16, 89],
  ['by', 17, 100],
  ['5', 18, 103],
  ['-', 19, 105],
  ['lipoxygenase', 20, 107],
  ['.', 21, 120]],
 [['IL - 2 gene', [0, 1, 2, 3], 'DNA'],
  ['NF - kappa B', [6, 7, 8, 9], 'protein'],
  ['CD28', [12], 'protein'],
  ['5 - lipoxygenase', [18, 19, 20], 'protein']]]

In [50]:
def gravaArquivoQA(dic_sentences, name):
    tipos= ['DNA','RNA','cell_type','cell_line','protein']

    texto=r'{"data": ['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesDNA=list()
        listaEntidadesRNA=list()
        listaEntidadesCell_type=list()
        listaEntidadesCell_line=list()
        listaEntidadesProtein=list()


        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'DNA':#problema
                listaEntidadesDNA.append(entidade)
            elif entidade[2] == 'RNA': #teste
                listaEntidadesRNA.append(entidade)
            elif entidade[2] == 'cell_type':#tratamento
                listaEntidadesCell_type.append(entidade)
            elif entidade[2] == 'cell_line':#anatomia
                listaEntidadesCell_line.append(entidade)
            elif entidade[2] == 'protein':
                listaEntidadesProtein.append(entidade)

        if len(listaEntidadesDNA)==0 and len(listaEntidadesRNA)==0 and len(listaEntidadesCell_type)==0 and len(listaEntidadesCell_line)==0 and len(listaEntidadesProtein)==0:
            continue

        title='texto_'+str(i)
        if num==0:
            texto = texto+'{"title":"'+title+'","paragraphs": [{"context": "'
        else:
            texto = texto[:len(texto)-1]
            texto = texto+']}]}, {"title":"'+title+'","paragraphs": [{"context": "'
        
        num=num+1

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        #contexto = contexto.replace(' , ', ', ').replace(' : ', ': ').replace(' . ', '. ').replace(' / ', '/ ')
        if '"' in contexto:
            print('tem aspassssss')
        texto = texto+contexto+'", "qas": ['


        if len(listaEntidadesDNA)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesDNA:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'
                

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "DNA", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesRNA)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesRNA:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [j[2] for j in dic_sentences[i][0] if j[1]==start][0]
                except:
                    print()
                    raise
                fim_string = [j[2] for j in dic_sentences[i][0] if j[1]==fim][0]
                fim_string = fim_string+[len(j[0]) for j in dic_sentences[i][0] if j[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "RNA", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesCell_type)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesCell_type:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                except:
                    print('i:', i)
                    print('dic_sentences[i][0]:', dic_sentences[i][0])
                    raise
                
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Cell_type", "id":"'+str(idQa)+'"},'

        if len(listaEntidadesCell_line)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesCell_line:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                except:
                    print('i:', i)
                    print('dic_sentences[i][0]:', dic_sentences[i][0])
                    raise
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            # tirando ultima virgula
            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Cell_line", "id":"'+str(idQa)+'"},'
			
        if len(listaEntidadesProtein)>0:
            if '"' in entidade[0]:
                print('tem aspassssss22')
            idQa=idQa+1
            texto = texto+'{"answers": ['
            for entidade in listaEntidadesProtein:
                texto = texto+'{"answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                try:
                    start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                except:
                    print('i:', i)
                    print('dic_sentences[i][0]:', dic_sentences[i][0])
                    raise                
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+', "text": "'+entidade[0]+'"},'

            # tirando ultima virgula
            texto = texto[:len(texto)-1]
            texto = texto+'], "question": "Protein", "id":"'+str(idQa)+'"},'			

    texto = texto[:len(texto)-1]
    texto = texto+']}]}], "version": "1.1"}'   
    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA(dic_sentencesTest2, 'data_qa/genia_test.json')
gravaArquivoQA(dic_sentencesTrain2, 'data_qa/genia_train.json')
gravaArquivoQA(dic_sentencesDev2, 'data_qa/genia_dev.json')

tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss22
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss22
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss22
tem aspassssss
tem aspassssss
tem aspassssss22
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss
tem aspassssss22
tem aspassssss
tem aspassssss
tem aspassssss


In [51]:
dic_sentencesTrain2[13374]

[[['YM268', 0, 0],
  ['increases', 1, 6],
  ['the', 2, 16],
  ['glucose', 3, 20],
  ['uptake', 4, 28],
  [',', 5, 35],
  ['cell', 6, 37],
  ['differentiation', 7, 42],
  [',', 8, 58],
  ['and', 9, 60],
  ['mRNA', 10, 64],
  ['expression', 11, 69],
  ['of', 12, 80],
  ['glucose', 13, 83],
  ['transporter', 14, 91],
  ['in', 15, 103],
  ['3T3', 16, 106],
  ['-', 17, 110],
  ['L1', 18, 112],
  ['adipocytes', 19, 115],
  ['.', 20, 126]],
 [['3T3 - L1 adipocytes', [16, 17, 18, 19], 'cell_type'],
  ['glucose transporter', [13, 14], 'protein']]]

In [52]:
dic_sentencesTrain2[9554]

[[['Taken', 0, 0],
  ['together', 1, 6],
  [',', 2, 15],
  ['these', 3, 17],
  ['data', 4, 23],
  ['suggest', 5, 28],
  ['that', 6, 36],
  ['tTG', 7, 41],
  [',', 8, 45],
  ['acting', 9, 47],
  ['on', 10, 54],
  ['the', 11, 57],
  ['pRB', 12, 61],
  ['protein', 13, 65],
  [',', 14, 73],
  ['might', 15, 75],
  ['play', 16, 81],
  ['an', 17, 86],
  ['important', 18, 89],
  ['role', 19, 99],
  ['in', 20, 104],
  ['the', 21, 107],
  ['cell', 22, 111],
  ['progression', 23, 116],
  ['through', 24, 128],
  ['the', 25, 136],
  ['death', 26, 140],
  ['program', 27, 146],
  ['.', 28, 154]],
 [['tTG', [7], 'protein'],
  ['pRB', [12], 'protein'],
  ['pRB protein', [12, 13], 'protein']]]

In [53]:
dic_sentencesTrain2[5865]

[[['In', 0, 0],
  ['T', 1, 3],
  ['lymphocytes', 2, 5],
  [',', 3, 17],
  ['the', 4, 19],
  ['calcium', 5, 23],
  ['/', 6, 31],
  ['calmodulin', 7, 33],
  ['-', 8, 44],
  ['dependent', 9, 46],
  ['serine', 10, 56],
  ['/', 11, 63],
  ['threonine', 12, 65],
  ['phosphatase', 13, 75],
  [',', 14, 87],
  ['calcineurin', 15, 89],
  [',', 16, 101],
  ['plays', 17, 103],
  ['a', 18, 109],
  ['pivotal', 19, 111],
  ['role', 20, 119],
  ['in', 21, 124],
  ['transducing', 22, 127],
  ['membrane', 23, 139],
  ['-', 24, 148],
  ['associated', 25, 150],
  ['signals', 26, 161],
  ['to', 27, 169],
  ['the', 28, 172],
  ['nucleus', 29, 176],
  ['.', 30, 184]],
 [['T lymphocytes', [1, 2], 'cell_type'],
  ['calcium / calmodulin - dependent serine / threonine phosphatase',
   [5, 6, 7, 8, 9, 10, 11, 12, 13],
   'protein'],
  ['calcineurin', [15], 'protein']]]

In [54]:
dic_sentencesTest2[1]

[[['When', 0, 0],
  ['the', 1, 5],
  ['homeodomain', 2, 9],
  ['from', 3, 21],
  ['HB24', 4, 26],
  ['was', 5, 31],
  ['compared', 6, 35],
  ['to', 7, 44],
  ['known', 8, 47],
  ['mammalian', 9, 53],
  ['and', 10, 63],
  ['Drosophila', 11, 67],
  ['homeodomains', 12, 78],
  ['it', 13, 91],
  ['was', 14, 94],
  ['found', 15, 98],
  ['to', 16, 104],
  ['be', 17, 107],
  ['only', 18, 110],
  ['moderately', 19, 115],
  ['conserved', 20, 126],
  [',', 21, 136],
  ['but', 22, 138],
  ['when', 23, 142],
  ['it', 24, 147],
  ['was', 25, 150],
  ['compared', 26, 154],
  ['to', 27, 163],
  ['a', 28, 166],
  ['highly', 29, 168],
  ['diverged', 30, 175],
  ['Drosophila', 31, 184],
  ['homeodomain', 32, 195],
  [',', 33, 207],
  ['H2', 34, 209],
  ['.', 35, 212],
  ['0', 36, 214],
  [',', 37, 216],
  ['it', 38, 218],
  ['was', 39, 221],
  ['found', 40, 225],
  ['to', 41, 231],
  ['be', 42, 234],
  ['80', 43, 237],
  ['%', 44, 240],
  ['identical', 45, 242],
  ['.', 46, 252]],
 [['homeodomain', [2],

In [179]:
import json 

try:
  file = r'data_qa/genia_train.json'
  # Opening JSON file & returns JSON object as a dictionary 
  #f = open(file, encoding="utf8") 
  f = open(file, encoding="utf8") 
  data = json.load(f) 
except:
  print('nao abriu arquivo:', file)
  print(data)
  raise

In [39]:
# versão 2 do squad
def gravaArquivoQA_impossible(dic_sentences, name):
    print('name:', name)
    tipos= ['DNA','RNA','Cell_type','Cell_line','Protein']

    texto=r'{"version": "v2.0", "data": ['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesDNA=list()
        listaEntidadesRNA=list()
        listaEntidadesCell_type=list()
        listaEntidadesCell_line=list()
        listaEntidadesProtein=list()


        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'DNA':#problema
                listaEntidadesDNA.append(entidade)
            elif entidade[2] == 'RNA': #teste
                listaEntidadesRNA.append(entidade)
            elif entidade[2] == 'cell_type':#tratamento
                listaEntidadesCell_type.append(entidade)
            elif entidade[2] == 'cell_line':#anatomia
                listaEntidadesCell_line.append(entidade)
            elif entidade[2] == 'protein':
                listaEntidadesProtein.append(entidade)

        if len(listaEntidadesDNA)==0 and len(listaEntidadesRNA)==0 and len(listaEntidadesCell_type)==0 and len(listaEntidadesCell_line)==0 and len(listaEntidadesProtein)==0:
            continue

        title='texto_'+str(i)
        if num==0:
            texto = texto+'{"title":"'+title+'","paragraphs": [{"qas": ['
        else:
            texto = texto[:len(texto)-1]
            texto = texto+'}]}, {"title":"'+title+'","paragraphs": [{"qas": ['
        
        num=num+1

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        
        
        if len(listaEntidadesDNA)>0:
            if '"' in entidade[0]:
                print('tem aspas em DNA')
            idQa=idQa+1
            texto = texto+'{"question": "DNA", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesDNA:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "DNA", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesRNA)>0:
            if '"' in entidade[0]:
                print('tem aspas em RNA')
            idQa=idQa+1
            texto = texto+'{"question": "RNA", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesRNA:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "RNA", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesCell_type)>0:
            if '"' in entidade[0]:
                print('tem aspas Cell_type')
            idQa=idQa+1
            texto = texto+'{"question": "Cell_type", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesCell_type:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Cell_type", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'

        if len(listaEntidadesCell_line)>0:
            if '"' in entidade[0]:
                print('tem aspas em Cell_line')
            idQa=idQa+1
            texto = texto+'{"question": "Cell_line", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesCell_line:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Cell_line", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'   
        
        if len(listaEntidadesProtein)>0:
            if '"' in entidade[0]:
                print('tem aspas em Protein')
            idQa=idQa+1
            texto = texto+'{"question": "Protein", "id":"'+str(idQa)+'", "answers": ['
            
            for entidade in listaEntidadesProtein:
                #print(dic_sentencesTest[i][0])
                texto = texto+'{"text":"'+entidade[0]+'", "answer_start":'
                indices = entidade[1]
                start =indices[0]
                fim = indices[-1]
                start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
                fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
                fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]

                texto = texto+str(start_string)+'},'
                #"is_impossible": false
                
            texto = texto[:len(texto)-1]
            texto = texto + '], "is_impossible": false},'
        
        else:
            idQa=idQa+1
            texto = texto+'{"question": "Protein", "id":"'+str(idQa)+'", "answers": [], "is_impossible": true},'
            
        texto = texto[:len(texto)-1]
        texto = texto+'], "context":"'+contexto+'"}' 
        
    texto = texto+']}]}'

    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA_impossible(dic_sentencesTest2, 'data_qa_impossible/genia_test_impossible.json')
gravaArquivoQA_impossible(dic_sentencesTrain2, 'data_qa_impossible/genia_train_impossible.json')
gravaArquivoQA_impossible(dic_sentencesDev2, 'data_qa_impossible/genia_dev_impossible.json')

name: data_qa_impossible/genia_test_impossible.json
name: data_qa_impossible/genia_train_impossible.json
tem aspas em Cell_line
tem aspas em DNA
tem aspas em Protein
tem aspas em Protein
name: data_qa_impossible/genia_dev_impossible.json
tem aspas em Protein


# gerando agora no formato do MRC

In [ ]:
# versão MRC

def gravaArquivoQA_mrc(dic_sentences, name):
    print('name:', name)
    tipos= ['DNA','RNA','Cell_type','Cell_line','Protein']


    texto=r'['
    # title
    # paragraphs
        #context, qas (answers (answer_start_original, text_original)), question (id, question_original)
    num=0
    idQa=1000000

    for i in range(len(dic_sentences)):
        ents = dic_sentences[i][1]

        listaEntidadesDNA=list()
        listaEntidadesRNA=list()
        listaEntidadesCell_type=list()
        listaEntidadesCell_line=list()
        listaEntidadesProtein=list()
        

        #print(contexto)
        for entidade in ents:
            if entidade[2] == 'DNA':#problema
                listaEntidadesDNA.append(entidade)
            elif entidade[2] == 'RNA': #teste
                listaEntidadesRNA.append(entidade)
            elif entidade[2] == 'Cell_type':#tratamento
                listaEntidadesCell_type.append(entidade)
            elif entidade[2] == 'Cell_line':#anatomia
                listaEntidadesCell_line.append(entidade)
            elif entidade[2] == 'Protein':
                listaEntidadesProtein.append(entidade)

                
                
        if len(listaEntidadesDNA)==0 and len(listaEntidadesRNA)==0 and len(listaEntidadesCell_type)==0 and len(listaEntidadesCell_line)==0 and len(listaEntidadesProtein)==0:
            continue

        tokens = dic_sentences[i][0]
        frase=[t[0] for t in tokens]
        contexto = ' '.join(frase)
        
        texto = texto+'{"context":"'+contexto+'",' 
        
        tipo=''
        #if len(listaEntidadesProblema)>0:
        tipo='DNA'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesDNA:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
            fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
            fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]
            #starts = [6,13]
            starts.append(start_string)
            ends.append(fim_string)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        
        tipo='RNA'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesRNA:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
            fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
            fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]
            #starts = [6,13]
            starts.append(start_string)
            ends.append(fim_string)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        
        tipo='Cell_type'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesCell_type:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
            fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
            fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]
            #starts = [6,13]
            starts.append(start_string)
            ends.append(fim_string)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        
        tipo='Cell_line'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesCell_line:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
            fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
            fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]
            #starts = [6,13]
            starts.append(start_string)
            ends.append(fim_string)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 

        tipo='Protein'
        if '"' in entidade[0]:
            print('tem aspas em ', tipo)
        idQa=idQa+1

        ends=[]
        starts=[]

        for entidade in listaEntidadesProtein:
            indices = entidade[1]
            start =indices[0]
            fim = indices[-1]
            start_string = [i[2] for i in dic_sentences[i][0] if i[1]==start][0]
            fim_string = [i[2] for i in dic_sentences[i][0] if i[1]==fim][0]
            fim_string = fim_string+[len(i[0]) for i in dic_sentences[i][0] if i[1]==fim][0]
            #starts = [6,13]
            starts.append(start_string)
            ends.append(fim_string)

        spans_pos=''
        impossible="true"
        spans_pos = '[]'
        if len(starts)>0:
            impossible="false"
            spans_pos = '['
            for a, b in zip(starts, ends):
                spans_pos = spans_pos +'"'+ str(a)+';'+str(b)+'",'
            spans_pos =spans_pos[0:-1]+ ']'


        texto = texto+'"end_position": '+str(ends)+', "entity_label":"'+tipo+'", "impossible":'+impossible+', "qas_id": "'+ str(idQa)+'",'
        texto = texto+'"query":"'+tipo+'", "span_position":'+str(spans_pos)+', "start_position":'+str(starts)+"}," 
       
        
    #texto = texto+']'
    texto =texto[0:-1]+ ']'

    arq = open(name, 'w', encoding='utf8')
    arq.write(texto)
    arq.close()
    #print(texto)
  
gravaArquivoQA_mrc(dic_sentencesTest2, 'data_qa_mrc/genia_test_mrc.json')
#gravaArquivoQA_mrc(dic_sentencesTrain2, 'data_qa_mrc/genia_train_mrc.json')
#gravaArquivoQA_mrc(dic_sentencesDev2, 'data_qa_mrc/genia_dev_mrc.json')